# Testing frameworks

## Why use testing frameworks?

Frameworks should simplify our lives:

* Should be easy to add simple test
* Should be possible to create complex test:
    * Fixtures
    * Setup/Tear down
    * Parameterized tests (same test, mostly same input)
* Find all our tests in a complicated code-base 
* Run all our tests with a quick command
* Run only some tests, e.g. ``test --only "tests about fields"``
* **Report failing tests**
* Additional goodies, such as code coverage

## Common testing frameworks

* Language agnostic: [CTest](http://www.cmake.org/cmake/help/v2.8.12/ctest.html)
  * Test runner for executables, bash scripts, etc...
  * Great for legacy code hardening
    
* C unit-tests:
    * all c++ frameworks,
    * [Check](http://check.sourceforge.net/),
    * [CUnit](http://cunit.sourceforge.net)

* C++ unit-tests:
    * [CppTest](http://cpptest.sourceforge.net/),
    * [Boost::Test](http://www.boost.org/doc/libs/1_55_0/libs/test/doc/html/index.html),
    * [google-test](https://code.google.com/p/googletest/),
    * [Catch](https://github.com/philsquared/Catch) (best)

* Python unit-tests:
    * [nose](https://nose.readthedocs.org/en/latest/) includes test discovery, coverage, etc
    * [unittest](http://docs.python.org/2/library/unittest.html) comes with standard python library
    * [py.test](http://pytest.org/latest/), branched off of nose

* R unit-tests:
    * [RUnit](http://cran.r-project.org/web/packages/RUnit/index.html),
    * [svUnit](http://cran.r-project.org/web/packages/svUnit/index.html)
    * (works with [SciViews](http://www.sciviews.org/) GUI)

* Fortran unit-tests:
    * [funit](http://nasarb.rubyforge.org/funit/),
    * [pfunit](http://sourceforge.net/projects/pfunit/)(works with MPI)

## py.test framework: usage

[py.test](https://docs.pytest.org/en/latest/) is a recommended python testing framework.

We can use its tools in the notebook for on-the-fly tests in the notebook. This, happily, includes the negative-tests example we were looking for a moment ago.

In [5]:
def I_only_accept_positive_numbers(number):
    # Check input
    if number < 0:
        raise ValueError("Input " + str(number) + " is negative")

    # Do something

In [6]:
from pytest import raises

In [7]:
with raises(ValueError):
    I_only_accept_positive_numbers(-5)

but the real power comes when we write a test file alongside our code files in our homemade packages:

In [35]:
%%bash
#on windows replace '%%bash' with %%cmd
rm -f saskatchewan
mkdir -p saskatchewan
# touch saskatchewan/__init__.py #on windows replace with 'type nul > saskatchewan/__init__.py'
type nul > saskatchewan/__init__.py

Couldn't find program: 'bash'


In [47]:
%%cmd   # or ! before every command
#on windows replace '%%bash' with %%cmd
rmdir /s saskatchewan
mkdir -p saskatchewan
## touch saskatchewan/__init__.py #on windows replace with 'type nul > saskatchewan/__init__.py'
type nul > saskatchewan/__init__.py
dir

Microsoft Windows [Version 10.0.19042.1466]
(c) Microsoft Corporation. All rights reserved.

(rse_course_2022) C:\Users\dmassegur\Projects\P001 - RSE Course\Materials\rse-course\module05_testing_your_code>#on windows replace '%%bash' with %%cmd

(rse_course_2022) C:\Users\dmassegur\Projects\P001 - RSE Course\Materials\rse-course\module05_testing_your_code>rmdir /s saskatchewan
saskatchewan, Are you sure (Y/N)? mkdir -p saskatchewan
saskatchewan, Are you sure (Y/N)? ## touch saskatchewan/__init__.py #on windows replace with 'type nul > saskatchewan/__init__.py'
saskatchewan, Are you sure (Y/N)? type nul > saskatchewan/__init__.py
saskatchewan, Are you sure (Y/N)? dir
saskatchewan, Are you sure (Y/N)? 

(rse_course_2022) C:\Users\dmassegur\Projects\P001 - RSE Course\Materials\rse-course\module05_testing_your_code>

'#on' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
%%writefile saskatchewan/overlap.py
def overlap(field1, field2):
    left1, bottom1, top1, right1 = field1
    left2, bottom2, top2, right2 = field2

    overlap_left = max(left1, left2)
    overlap_bottom = max(bottom1, bottom2)
    overlap_right = min(right1, right2)
    overlap_top = min(top1, top2)
    # Here's our wrong code again
    overlap_height = overlap_top - overlap_bottom
    overlap_width = overlap_right - overlap_left

    return overlap_height * overlap_width

Overwriting saskatchewan/overlap.py


In [20]:
%%writefile saskatchewan/test_overlap.py
from .overlap import overlap


def test_full_overlap():
    assert overlap((1.0, 1.0, 4.0, 4.0), (2.0, 2.0, 3.0, 3.0)) == 1.0


def test_partial_overlap():
    assert overlap((1, 1, 4, 4), (2, 2, 3, 4.5)) == 2.0


def test_no_overlap():
    assert overlap((1, 1, 4, 4), (4.5, 4.5, 5, 5)) == 0.0

Writing saskatchewan/test_overlap.py


In [59]:
%%cmd #(windows)    %%bash (linux)
dir
!chdir saskatchewan
dir
py.test || echo "Tests failed"

Microsoft Windows [Version 10.0.19042.1466]
(c) Microsoft Corporation. All rights reserved.

(rse_course_2022) C:\Users\dmassegur\Projects\P001 - RSE Course\Materials\rse-course\module05_testing_your_code>dir
 Volume in drive C is OS
 Volume Serial Number is 4CD9-CA7E

 Directory of C:\Users\dmassegur\Projects\P001 - RSE Course\Materials\rse-course\module05_testing_your_code

21/01/2022  15:03    <DIR>          .
21/01/2022  15:03    <DIR>          ..
21/01/2022  14:12    <DIR>          -p
21/01/2022  14:27    <DIR>          .ipynb_checkpoints
21/01/2022  14:13    <DIR>          .pytest_cache
21/01/2022  13:13             4,671 05_00_introduction.ipynb
21/01/2022  13:49            58,443 05_01_how_to_test.ipynb
21/01/2022  15:03            24,074 05_02_testing_frameworks.ipynb
21/01/2022  14:57            24,203 05_03_energy_example.ipynb
17/01/2022  13:45            77,160 05_04_mocking.ipynb
17/01/2022  13:45             7,075 05_05_using_a_debugger.ipynb
17/01/2022  13:45           

'!chdir' is not recognized as an internal or external command,
operable program or batch file.


Note that it reported **which** test had failed, how many tests ran, and how many failed.

The symbol `..F` means there were three tests, of which the third one failed.

Pytest will:

* automagically finds files ``test_*.py``
* collects all subroutines called ``test_*``
* runs tests and reports results

Some options:

* help: `py.test --help`
* run only tests for a given feature: `py.test -k foo` # tests with 'foo' in the test name

# Testing with floating points

## Floating points are not reals


Floating points are inaccurate representations of real numbers:

`1.0 == 0.99999999999999999` is true to the last bit.

This can lead to numerical errors during calculations: $1000 (a - b) \neq 1000a - 1000b$

In [13]:
1000.0 * 1.0 - 1000.0 * 0.9999999999999998

2.2737367544323206e-13

In [14]:
1000.0 * (1.0 - 0.9999999999999998)

2.220446049250313e-13

*Both* results are wrong: `2e-13` is the correct answer.

The size of the error will depend on the magnitude of the floating points:

In [16]:
1000.0 * 1e5 - 1000.0 * 0.9999999999999998e5

1.4901161193847656e-08

The result should be `2e-8`.

## Comparing floating points

Use the "approx", for a default of a relative tolerance of $10^{-6}$

In [23]:
from pytest import approx

assert 0.7 == approx(0.7 + 1e-7)

Or be more explicit:

In [24]:
magnitude = 0.7
assert 0.7 == approx(0.701, rel=0.1, abs=0.1)

Choosing tolerances is a big area of debate: https://software-carpentry.org/blog/2014/10/why-we-dont-teach-testing.html

## Comparing vectors of floating points

Numerical vectors are best represented using [numpy](http://www.numpy.org/).

In [25]:
from numpy import array, pi

vector_of_reals = array([0.1, 0.2, 0.3, 0.4]) * pi

Numpy ships with a number of assertions (in ``numpy.testing``) to make
comparison easy:

In [28]:
from numpy import array, pi
from numpy.testing import assert_allclose

expected = array([0.1, 0.2, 0.3, 0.4, 1e-12]) * pi
actual = array([0.1, 0.2, 0.3, 0.4, 2e-12]) * pi
actual[:-1] += 1e-6
assert_allclose(actual, expected, rtol=1e-5, atol=1e-13)  ## elementwise comparison within tolerance

AssertionError: 
Not equal to tolerance rtol=1e-05, atol=1e-13

Mismatched elements: 1 / 5 (20%)
Max absolute difference: 1.e-06
Max relative difference: 1.
 x: array([3.141603e-01, 6.283195e-01, 9.424788e-01, 1.256638e+00,
       6.283185e-12])
 y: array([3.141593e-01, 6.283185e-01, 9.424778e-01, 1.256637e+00,
       3.141593e-12])

It compares the difference between `actual` and `expected` to ``atol + rtol * abs(expected)``.